<a href="https://colab.research.google.com/github/Navaneeth-18bit/Study-Planner-AI-chatbot/blob/main/study_planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers

In [ ]:
from transformers import pipeline
import re

llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    max_length=128
)

print("Agentic AI (type 'exit' to quit)")

def extract_subjects(text):
  prompt = f"""
  Extract the study subjects mentioned in the following text.
  Return ONLY a comma-seperated list.
  No explanation.

  Text:
  {user_input}
  """

  result = llm(prompt)[0]["generated_text"]
  subjects = [s.strip() for s in result.split(",") if s.strip()]
  return subjects


def allocate_study_time(subjects, total_hours=4):
  """simple logic: split hours evenly among subjects
  """

  n = len(subjects)
  per_subject = round(total_hours/n,1)
  plan = {subject: per_subject for subject in subjects}
  return plan

def format_plan(plan):
  lines=[]
  for subject, hours in plan.items():
    lines.append(f"={subject}: {hours} hours")
  return "\n".join(lines)

def chat_tool(user_input):

  subjects = extract_subjects(user_input)

  if not subjects:
    return "I couldn't detect sunjects. Please mention them clearly"

  plan = allocate_study_time(subjects)
  formatted_plan = format_plan(plan)



  prompt_chat = f"""
  You are a study planner assistant


  Study Plan:
  {formatted_plan}

  Now Explain this plan clearly.
  Give step-by-step guidance.
  Add time managment tips.


  """
  return llm(prompt_chat)[0]["generated_text"].strip()


while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Agent: Goodbye!")
        break

    result = chat_tool(user_input)
    print("Agent:", result)



Device set to use cpu


Agentic AI (type 'exit' to quit)
You: I have two subjects DBMS and OS
Agent: You are a study planner assistant Study Plan: =DBMS: 2.0 hours = OS: 2.0 hours Now Explain this plan clearly. Give step-by-step guidance. Add time managment tips. You are a study planner assistant Study Plan: =DBMS: 2.0 hours = OS: 2.0 hours Now Explain this plan clearly. Give step-by-step guidance. Add time managment tips. You are a study planner assistant Study Plan: =DBMS: 2.0 hours = OS: 2.0 hours Now Explain this plan clearly. Give step-by-step guidance. Add time managment
You: exit
Agent: Goodbye!


In [ ]:
from transformers import pipeline
import re
import datetime

llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    max_length=128
)

print("Agentic AI (type 'exit' to quit)")


TIME_KEYWORDS = ["hour","hours","time","schedule","plan","split","manage","study time"]

MATH_KEYWORDS = ["+","-","*","/"]

STUDY_INTENT = ["study","plan","schedule","timetable","prepare"]

MEMORY = {
    "time_minutes": None,
    "subjects": None,
    "chapters":None
}


SUBJECT_KEYWORDS = [
    # Core subjects
    "dbms", "os", "cn", "ai", "ml",
    "maths", "physics", "chemistry",
    "english", "history", "biology",

    # Study intent words
    "study", "studying", "learn", "learning",
    "revise", "revision", "prepare", "preparation",
    "plan", "schedule", "timetable",
    "subject", "syllabus", "exam"
]




def calculator_tool(text):
    """Calculate simple math expressions."""
    try:
        # Extract numbers and operators
        expression = re.findall(r"[0-9]+|[\+\-\*/]", text)
        return str(eval("".join(expression)))
    except:
        return "I couldn't calculate that."

def time_managment_tool(text):
    global MEMORY

    # -------- TIME --------
    hours = re.findall(r"\d+", text)
    if hours:
        MEMORY["time_minutes"] = int(hours[0]) * 60

    if MEMORY["time_minutes"] is None:
        return "How much total time do you have?"

    total_minutes = MEMORY["time_minutes"]

    # -------- SUBJECTS --------
    subjects = re.findall(r"(DBMS|OS|CN|AI|ML)", text, re.IGNORECASE)
    if subjects:
        MEMORY["subjects"] = subjects

    if MEMORY["subjects"] is None:
        return "Which subjects should I plan for?"

    subjects = MEMORY["subjects"]
    subject_count = len(subjects)

    # -------- CHAPTERS --------
    chapter_match = re.search(r"(\d+)\s*chapters?", text)
    if chapter_match:
        MEMORY["chapters"] = int(chapter_match.group(1))
    elif MEMORY["chapters"] is None:
        MEMORY["chapters"] = 1   # default

    chapters = MEMORY["chapters"]

    # -------- STUDY TIME --------
    study_time = total_minutes // subject_count

    if study_time < 30:
        break_time = 0
    elif study_time <= 60:
        break_time = 5
    else:
        break_time = 10

    chapter_time = study_time // chapters

    # -------- RESPONSE --------
    response = []
    response.append("📘 STUDY PLAN\n")
    response.append(f"You have {subject_count} subject(s) to cover")
    response.append(f"Total chapters: {chapters}\n")

    for subject in subjects:
        response.append(
            f"📚 {subject.upper()}\n"
            f"• Total study time: {study_time} minutes\n"
            f"• Study each chapter for {chapter_time} minutes\n"
            f"• Take a {break_time}-minute break after each chapter or after 1 or 1.5 hours\n"
        )

    response.append(
        "💡 Tips:\n"
        "• Focus on understanding, not memorizing\n"
        "• Revise after every chapter\n"
        "• Small progress every day builds confidence\n"
        "• You are doing great — keep going 💪"
    )

    return "\n".join(response)




def clarification_response(text):

  """This is the agent asks clarification for the study plan response"""

  global MEMORY

  if MEMORY["time_minutes"] is None:
    return "How much total time do you have"

  if MEMORY["subjects"] is None:
    return "Which subjects should i plan for?"

  return "Please give more details."

  needs_time = not re.search(r"\d+", text)
  needs_subject = not any(word in text for word in SUBJECT_KEYWORDS)
  if needs_time and needs_subject:
      return "How much total time do you have and which subjects should I plan for?"
  elif needs_time:
      return "How much total time do you have?"
  elif needs_subject:
      return "Which subjects should I plan for?"
  else:
      return "Can you give more details?"

    #-------CHAT TOOL--------

def chat_tool():
  prompt_chat = f"""
Define the following term or answer the question clearly.

Rules:
- Give a correct definition or explanation
- Use simple English
- Do NOT repeat the question
- Do NOT output instructions

Input: {user_input}
Output:
"""
  return llm(prompt_chat)[0]["generated_text"].strip()




def decide_and_act(user_input):

    has_math = any(op in user_input for op in MATH_KEYWORDS)
    has_number = bool(re.search(r"\d+", user_input))
    has_time_word = any(word in user_input for word in TIME_KEYWORDS)
    has_subject = any(word in user_input for word in SUBJECT_KEYWORDS)
    has_study_intent = any(word in user_input for word in STUDY_INTENT)

    # ---- MULTI TOOL ----
    if has_math and has_study_intent:
        calc = calculator_tool(user_input)
        return time_managment_tool(user_input + " " + calc)

    # ---- MATH ----
    if has_math:
        return calculator_tool(user_input)

    # ---- STUDY PLANNING ----
    if has_study_intent:
        if has_number and has_time_word and has_subject:
            return time_managment_tool(user_input)
        else:
            return clarification_response(user_input)

    # ---- NORMAL CHAT ----
    return chat_tool()
def normalize_input(text):
  """Normalize the input of the prompt"""
  text = text.lower()
  text = text.replace("hrs","hours")
  text = text.replace("hr","hour")
  text = text.replace("mins","minutes")
  text = text.replace("min","miunte")
  return text



while True:

  raw_input = input("You:")

  if raw_input.lower() == "exit":
    print("Agent: Goodbye!")
    break



  user_input = normalize_input(raw_input)

  result = decide_and_act(user_input)
  print("Agent:", result)


Device set to use cpu


Agentic AI (type 'exit' to quit)
You:say helloo
Agent: Say helloo
You:make a study plan for DBMS 7 hrs
Agent: 📘 STUDY PLAN

You have 1 subject(s) to cover
Total chapters: 1

📚 DBMS
• Total study time: 420 minutes
• Study each chapter for 420 minutes
• Take a 10-minute break after each chapter or after 1 or 1.5 hours

💡 Tips:
• Focus on understanding, not memorizing
• Revise after every chapter
• Small progress every day builds confidence
• You are doing great — keep going 💪
You:who is Mahathma Ghandhi?
Agent: Mahatma Ghandhi (, also Romanized as Mhathm Gdh; also known as Mhathm Gdh; also known as Mhathm Gdh; also known as Mhathm Gdh; also known as Mhathm Gdh; also known as Mhathm Gdh; also known as Mhathm Gdh; also known as Mhathm G
You:What is Normalization?
Agent: Normalization is the process of determining the normal distribution of a variable.
You:who is APJ Abdul Kalam?
Agent: APJ Abdul Kalam is a Pakistani politician.
You:Who is APJ Abdul Kalam?
Agent: APJ Abdul Kalam is a Pakist